# Earthquake Damage

We're trying to predict the ordinal variable damage_grade, which represents a level of damage to the building that was hit by the earthquake. There are 3 grades of the damage:

* 1 represents low damage
* 2 represents a medium amount of damage
* 3 represents almost complete destruction

The data was collected through surveys by Kathmandu Living Labs and the Central Bureau of Statistics, which works under the National Planning Commission Secretariat of Nepal. This survey is one of the largest post-disaster datasets ever collected, containing valuable information on earthquake impacts, household conditions, and socio-economic-demographic statistics.

We are predicting the level of damage from 1 to 3. The level of damage is an ordinal variable meaning that ordering is important. This can be viewed as a classification or an ordinal regression problem. (Ordinal regression is sometimes described as an problem somewhere in between classification and regression.)

# Libraries

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler

In [2]:
# Import metrics
from sklearn.metrics import (accuracy_score, 
                            f1_score, 
                            confusion_matrix, 
                            classification_report,
                            confusion_matrix)

In [3]:
# Import models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR

# Data

In [4]:
# Load the labels
labels = pd.read_csv('data/train_labels.csv')
labels.head()

,building_id,damage_grade
0,802906,3
1,28830,2
2,94947,3
3,590882,2
4,201944,3


In [5]:
# Check out the size
labels.shape

(260601, 2)

In [6]:
# Load the values
values = pd.read_csv('data/train_values.csv')
values.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Check out the shape
values.shape

(260601, 39)

In [8]:
# Merge labels and values into on dataframe
df = pd.merge(labels, values)
df.head()

# Why does concat never work for me?
# df = pd.concat([labels, values], axis=1)

,building_id,damage_grade,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,3,6,487,12198,2,30,6,5,t,...,0,0,0,0,0,0,0,0,0,0
1,28830,2,8,900,2812,2,10,8,7,o,...,0,0,0,0,0,0,0,0,0,0
2,94947,3,21,363,8973,2,10,5,5,t,...,0,0,0,0,0,0,0,0,0,0
3,590882,2,22,418,10694,2,10,6,5,t,...,0,0,0,0,0,0,0,0,0,0
4,201944,3,11,131,1488,3,30,8,9,t,...,0,0,0,0,0,0,0,0,0,0


# Explore

In [ ]:
# Total df shape
df.shape

In [ ]:
# Basic info
df.info()

In [ ]:
# Again we see there aren't any missing values
df.isnull().sum()

In [ ]:
# Continuous features stats
df.describe()

In [ ]:
# Look at the damage_grades - looks pretty unbalanced
import seaborn as sns

damange_count = df['damage_grade'].value_counts()
plt.figure(figsize=(10,5))
sns.barplot(x=damange_count.index, y=damange_count.values, palette='copper')
plt.title('Damage by Grade', fontsize=18)
plt.ylabel('Number of Buildings', fontsize=12)
plt.xlabel('Grade', fontsize=12)
plt.legend(labels=["1 Low Damage","2 Medium Damage","3 Complete Destruction"])
plt.show()

* 1 represents low damage
* 2 represents a medium amount of damage
* 3 represents almost complete destruction

In [ ]:
# Age of buildings is going to be an issue.
plt.figure(figsize=(10,5))
sns.histplot(data=df, x="age", bins=20, color='#93684b')
plt.title('Age of Buildings', fontsize=18)
plt.ylabel('Number of Buildings', fontsize=12)
plt.xlabel('Age', fontsize=12)
plt.show()

In [ ]:
df.age.mean()

In [ ]:
df.age.max()

In [ ]:
df.age.std()

In [ ]:
df.age.std()*3

In [ ]:
df.age.mean() + df.age.std()*3

In [ ]:
sum(i > (df.age.mean() + df.age.std()*3) for i in df.age)

In [ ]:
df.loc[df['age'] > (df.age.mean() + df.age.std()*3)]

In [ ]:
old_structures = df.loc[df['age'] > (df.age.mean() + df.age.std()*3)]

In [ ]:
old_structures['damage_grade'].value_counts()

In [ ]:
df2 = df.reset_index().groupby(['age', 'damage_grade']).size().to_frame('count')
df2['percentage'] = df2['count'].div(df2.groupby('age')['count'].transform('sum')).mul(100)

In [ ]:
df2

In [ ]:
# Age of buildings is going to be an issue.
plt.figure(figsize=(10,5))
sns.histplot(data=df, x="height_percentage", bins=20, color='#93684b')
plt.title('Height Percentage', fontsize=18)
plt.ylabel('Number of Buildings', fontsize=12)
plt.xlabel('height_percentage', fontsize=12)
plt.show()

In [ ]:
# Floor count.
plt.figure(figsize=(10,5))
sns.histplot(data=df, x="count_floors_pre_eq", bins=20, color='#93684b')
plt.title('Height Percentage', fontsize=18)
plt.ylabel('Number of Buildings', fontsize=12)
plt.xlabel('count_floors_pre_eq', fontsize=12)
plt.show()

In [ ]:
df.count_floors_pre_eq.value_counts()

In [ ]:
df3 = df.reset_index().groupby(['count_floors_pre_eq', 'damage_grade']).size().to_frame('count')
df3['percentage'] = df3['count'].div(df3.groupby('count_floors_pre_eq')['count'].transform('sum')).mul(100)

In [ ]:
df3

In [ ]:
objects = ['land_surface_condition','foundation_type','roof_type','ground_floor_type',
           'other_floor_type','position','plan_configuration','legal_ownership_status']

In [ ]:
# One-hot encoding objects is going to add 30 more columns (with drop_first = True)
total = 0
for obj in objects:
    print(df[obj].value_counts())
    num = df[obj].value_counts().count()
    total += num
total

In [ ]:
df.describe()

# Cleaning

Best Practices - split out a separate test set, tune hyperparameters, or implement cross-validation

# Imbalanced?

In [ ]:
df['damage_grade'].value_counts()

In [ ]:
# Create a dataset that is balanced!

# Separate majority and minority classes
df_majority = df[df.balance==0]
df_minority = df[df.balance==1]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=49,     # to match minority class
                                 random_state=123) # reproducible results

# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.balance.value_counts()
# 1    49
# 0    49
# Name: balance, dtype: int64

In [ ]:
# Create a dataset that is balanced!

# Separate majority and minority classes
df_majority = df[df.damage_grade == 2]
df_minority = df[df.damage_grade == 1]
df_middle = df[df.damage_grade == 3]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, replace=False, 
                                   n_samples=len(df[df.damage_grade == 3]), random_state=123)

# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_middle])
 
# Display new class counts
df_downsampled.damage_grade.value_counts()

In [ ]:
# Again...
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, replace=False, 
                                   n_samples=len(df[df.damage_grade == 3]), random_state=123)

# Combine minority class with downsampled majority class
df_new = pd.concat([df_minority_upsampled, df_downsampled])
 
# Display new class counts
df_new.damage_grade.value_counts()

# Train/Test

In [9]:
df.head()

,building_id,damage_grade,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,3,6,487,12198,2,30,6,5,t,...,0,0,0,0,0,0,0,0,0,0
1,28830,2,8,900,2812,2,10,8,7,o,...,0,0,0,0,0,0,0,0,0,0
2,94947,3,21,363,8973,2,10,5,5,t,...,0,0,0,0,0,0,0,0,0,0
3,590882,2,22,418,10694,2,10,6,5,t,...,0,0,0,0,0,0,0,0,0,0
4,201944,3,11,131,1488,3,30,8,9,t,...,0,0,0,0,0,0,0,0,0,0


In [10]:
y = df.damage_grade
y.head()

0    3
1    2
2    3
3    2
4    3
Name: damage_grade, dtype: int64

In [11]:
X = df.select_dtypes(exclude=['object'])
X = X.drop('damage_grade', axis = 1)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 0 to 260600
Data columns (total 31 columns):
 #   Column                                  Non-Null Count   Dtype
---  ------                                  --------------   -----
 0   building_id                             260601 non-null  int64
 1   geo_level_1_id                          260601 non-null  int64
 2   geo_level_2_id                          260601 non-null  int64
 3   geo_level_3_id                          260601 non-null  int64
 4   count_floors_pre_eq                     260601 non-null  int64
 5   age                                     260601 non-null  int64
 6   area_percentage                         260601 non-null  int64
 7   height_percentage                       260601 non-null  int64
 8   has_superstructure_adobe_mud            260601 non-null  int64
 9   has_superstructure_mud_mortar_stone     260601 non-null  int64
 10  has_superstructure_stone_flag           260601 non-null  int64
 11  

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

# Scale the Data

In [13]:
# MinMax Scaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Model

In [ ]:
#KNN Model
knn_clf = KNeighborsClassifier(random_state=123, n_neighbors=3)
knn_clf.fit(X_train, y_train)

knn_training_preds = knn_clf.predict(X_train)
knn_training_accuracy = accuracy_score(y_train, knn_training_preds)

knn_val_preds = knn_clf.predict(X_test)
knn_val_accuracy = accuracy_score(y_test, knn_val_preds)

print("KNN Training Accuracy: {:.4}%".format(knn_training_accuracy * 100))
print("KNN Validation accuracy: {:.4}%".format(knn_val_accuracy * 100))
print("KNN F1 Score: {:.4}%".format(f1_score(y_test, knn_val_preds, average='micro') * 100))

In [14]:
# Forest Model
forest_clf = RandomForestClassifier(random_state=123)
forest_model = forest_clf.fit(X_train, y_train)

forest_training_preds = forest_clf.predict(X_train)
forest_training_accuracy = accuracy_score(y_train, forest_training_preds)

forest_val_preds = forest_clf.predict(X_test)
forest_val_accuracy = accuracy_score(y_test, forest_val_preds)

print("Forest Training Accuracy: {:.4}%".format(forest_training_accuracy * 100))
print("Forest Validation accuracy: {:.4}%".format(forest_val_accuracy * 100))
print("Forest F1 Score: {:.4}%".format(f1_score(y_test, forest_val_preds, average='micro') * 100))

Forest Training Accuracy: 100.0%
Forest Validation accuracy: 71.57%
Forest F1 Score: 71.57%


In [ ]:
# Log model
log_clf = LogisticRegression(random_state=123, solver= 'newton-cg', max_iter=90)
log_model = log_clf.fit(X_train, y_train)

log_training_preds = log_clf.predict(X_train)
log_training_accuracy = accuracy_score(y_train, log_training_preds)

log_val_preds = log_clf.predict(X_test)
log_val_accuracy = accuracy_score(y_test, log_val_preds)

print("Log Training Accuracy: {:.4}%".format(log_training_accuracy * 100))
print("Log Validation Accuracy: {:.4}%".format(log_val_accuracy * 100))
print("Log F1 Score: {:.4}%".format(f1_score(y_test, log_val_preds, average='micro') * 100))

In [ ]:
# SVR model
svr_clf = SVR(random_state=123, kernel='linear', class_weight='balanced')
svr_model = svr_clf.fit(X_train, y_train)

svr_training_preds = svr_clf.predict(X_train)
svr_training_accuracy = accuracy_score(y_train, svr_training_preds)

svr_val_preds = svr_clf.predict(X_test)
svr_val_accuracy = accuracy_score(y_test, svr_val_preds)

print("SVR Training Accuracy: {:.4}%".format(svr_training_accuracy * 100))
print("SVR Validation Accuracy: {:.4}%".format(svr_val_accuracy * 100))
print("SVR F1 Score: {:.4}%".format(f1_score(y_test, svr_val_preds, average='micro') * 100))

In [ ]:
# Train model
clf_0 = LogisticRegression().fit(X, y)
 
# Predict on training set
pred_y_0 = clf_0.predict(X)

In [ ]:
# Baseline Logistic Regession Model

# Separate input features (X) and target variable (y)
y = df.balance
X = df.drop('balance', axis=1)
 
# Train model
clf_0 = LogisticRegression().fit(X, y)
 
# Predict on training set
pred_y_0 = clf_0.predict(X)

In [ ]:
# Using Support Vectors with "balance"

# Separate input features (X) and target variable (y)
y = df.balance
X = df.drop('balance', axis=1)
 
# Train model
clf_3 = SVC(kernel='linear', 
            class_weight='balanced', # penalize
            probability=True)
 
clf_3.fit(X, y)
 
# Predict on training set
pred_y_3 = clf_3.predict(X)
 
# Is our model still predicting just one class?
print( np.unique( pred_y_3 ) )
# [0 1]
 
# How's our accuracy?
print( accuracy_score(y, pred_y_3) )
# 0.688
 
# What about AUROC?
prob_y_3 = clf_3.predict_proba(X)
prob_y_3 = [p[1] for p in prob_y_3]
print( roc_auc_score(y, prob_y_3) )
# 0.5305236678

In [ ]:
# Random Forest - good for imbalanced datasets!

# Separate input features (X) and target variable (y)
y = df.balance
X = df.drop('balance', axis=1)
 
# Train model
clf_4 = RandomForestClassifier()
clf_4.fit(X, y)
 
# Predict on training set
pred_y_4 = clf_4.predict(X)
 
# Is our model still predicting just one class?
print( np.unique( pred_y_4 ) )
# [0 1]
 
# How's our accuracy?
print( accuracy_score(y, pred_y_4) )
# 0.9744
 
# What about AUROC?
prob_y_4 = clf_4.predict_proba(X)
prob_y_4 = [p[1] for p in prob_y_4]
print( roc_auc_score(y, prob_y_4) )
# 0.999078798186

# Conclusion

# Future Work

# Submission

In [25]:
# Imput test data
test_set = pd.read_csv('data/test_values.csv')
test_set.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,300051,17,596,11307,3,20,7,6,t,r,...,0,0,0,0,0,0,0,0,0,0
1,99355,6,141,11987,2,25,13,5,t,r,...,1,0,0,0,0,0,0,0,0,0
2,890251,22,19,10044,2,5,4,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,745817,26,39,633,1,0,19,3,t,r,...,0,0,1,0,0,0,0,0,0,0
4,421793,17,289,7970,3,15,8,7,t,r,...,0,0,0,0,0,0,0,0,0,0


Cleaning?

In [26]:
# Drop the non-linear columns
test = test_set.select_dtypes(exclude=['object'])

In [27]:
# Scale the test set
test = scaler.transform(test)

In [28]:
# Predict target values on test set
forest_test_preds = forest_clf.predict(test)

In [29]:
# Change array to a dataframe
forest_target = pd.DataFrame(forest_test_preds)
forest_target.head()

,0
0,3
1,2
2,2
3,1
4,3


In [32]:
# Put building_id back in dataframe
forest_output = pd.merge(test_set['building_id'], forest_target, how = 'left', 
                         left_index = True, right_index = True)

In [33]:
# Check it out
forest_output.head()

,building_id,0
0,300051,3
1,99355,2
2,890251,2
3,745817,1
4,421793,3


In [34]:
# Put in the column name
forest_output = forest_output.rename(columns = {0:"damage_grade"}) 
forest_output.head()

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,2
3,745817,1
4,421793,3


In [35]:
forest_output.to_csv('whipple_earthquake_forest_submission1.csv', index=False)

In [36]:
submission_attempt_1 = pd.read_csv('./whipple_earthquake_forest_submission1.csv')
submission_attempt_1.head()

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,2
3,745817,1
4,421793,3


![alt text](Images/earthquake_forest_submission_1.png "Submission Score 1")